# Ejemplo de clasificacion

## Importar librerias

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

## Exploracion Dataset

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth',300)

In [ ]:
# Importo el dataset
df = pd.read_csv('/content/drive/MyDrive/Austral/TextMining/Clases/Clase3/Practica/datos/archive.zip', encoding='utf-8') #Encoding
df.shape

In [ ]:
df.head(10)

In [ ]:
print("Cantidad de targets: ")
df.label.value_counts()

In [ ]:
# Creo una caracteristica con el tamaño del titulo
df['title_text_size'] = df['title'].str.len()
df['title_text_words_count'] = df['title'].str.split().apply(len)
df['text_text_size'] = df['text'].str.len()
df['text_text_words_count'] = df['text'].str.split().apply(len)

In [ ]:
import plotly.express as px
fig = px.histogram(df, x="title_text_words_count", color='label',histnorm='probability')
fig.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def get_top_n_words(corpus, n=10):
    cv = CountVectorizer() # Agregar StopWords
    corpus_matrix = cv.fit_transform(corpus)
    corpus_matrix = pd.DataFrame.sparse.from_spmatrix(corpus_matrix, columns=cv.get_feature_names())
    aux = corpus_matrix.sum()
    aux=aux.to_frame('count')
    aux.sort_values(by='count',ascending=False,inplace=True)
    aux=aux.head(n)
    return aux

In [ ]:
# Cantidad de tokens en todo el corpus
most_importants = get_top_n_words(df.text, n=500)
print(most_importants.head(10))
px.bar(most_importants, x=most_importants.index, y='count')

In [ ]:
# Cantidad de tokens en todo el corpus
most_importants = get_top_n_words(df[df.label=='FAKE'].text, n=500)
print(most_importants.head(10))
px.bar(most_importants, x=most_importants.index, y='count')

In [ ]:
# Cantidad de tokens en todo el corpus
most_importants = get_top_n_words(df[df.label=='REAL'].text, n=500)
print(most_importants.head(10))
px.bar(most_importants, x=most_importants.index, y='count')

## Proceso NLP

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
stop_words = set( stopwords.words('english'))
stop_words.update(string.punctuation)
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

# Pre procesamiento basico, agregar las etapas que considere necesario
def pre_procesamiento_texto(text):
  # Armo los tokens para procesar los datos
  tokens = word_tokenize(text)

  # Elimino las stopwords
  tokens = [t.lower() for t in tokens if t.lower() not in stop_words]

  return tokens

In [ ]:
text='Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam'
print(text)
pre_procesamiento_texto(text)

## Vectorizacion resultados

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.text, df.label,test_size=.30)

In [ ]:
# Vectorizacion de resultados BOW, tambien se puede probar TF-IDF
cv = CountVectorizer(tokenizer=pre_procesamiento_texto)
X_train_transform = cv.fit_transform(X_train) # Armo matriz para entrenar

In [ ]:
X_train_transform

In [ ]:
df_tranform = pd.DataFrame.sparse.from_spmatrix(X_train_transform, columns=cv.get_feature_names())
df_tranform

In [ ]:
aux = df_tranform.sum().to_frame('count')
aux.sort_values(by='count',ascending=False,inplace=True)
aux=aux.head(1000)
px.bar(aux, x=aux.index, y='count')

In [ ]:
print("Vocabulario de ", len(cv.get_feature_names()))

In [ ]:
X_train_transform.shape

Se puede ver que tenemos mas features que observaciones. Si se utiliza algun metodo de algebra lineal se necesitan tener al menos la misma cantidad de observaciones que de incognitas, sino existiran infinitas soluciones haciendo que el modelo no generalize correctamente. 

Es recomendable utilizar tecnicas de Cross 
Validation,  reduccion de dimencionalidad y seleccion de features.

## Modelo Regresion Logistica

In [ ]:
clf = LogisticRegression()
clf.fit(X_train_transform, y_train)

### Resultados obtenidos

In [ ]:
X_test_tranform = cv.transform(X_test) 
y_pred = clf.predict(X_test_tranform.toarray())

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import plot_roc_curve,plot_confusion_matrix
plot_roc_curve(clf,X_test_tranform,y_test)

In [ ]:
from sklearn.metrics import plot_roc_curve,plot_confusion_matrix
plot_confusion_matrix(clf,X_test_tranform,y_test,normalize='true')

In [ ]:
features = pd.DataFrame()
features['feature'] = cv.get_feature_names()
features['importance'] = clf.coef_.flatten()

features = features.sort_values(by='importance')

features_top_positivas = features[:200]
features_top_negativas = features[-200:]

px.bar(pd.concat([features_top_negativas, features_top_positivas]).sort_values(by='importance'), x='feature', y='importance')

## Modelo SVM

**Support vector machines** (SVM) son un conjunto de métodos de aprendizaje supervisados classification, regression y outliers detection.

Las ventajas de los SVM son:<br>
  * Efectivo en espacios de altas dimensiones.
  * Sigue siendo efectivo en casos donde el número de dimensiones es mayor que el número de muestras, pero dependera del nivel de regularizacion
  * Utiliza un subconjunto de puntos de entrenamiento en la función de decisión (llamados vectores de soporte), por lo que también es eficiente en la memoria.
  * Versátil: se pueden especificar diferentes funciones de Kernel para la función de decisión. Se proporcionan núcleos comunes, pero también es posible especificar núcleos personalizados.

Las desventajas de los SVM son:<br>
  * Si el número de características es mucho mayor que el número de muestras, se necesita evitar el ajuste excesivo al elegir las funciones de Kernel y el término de regularización es crucial.
  * Los SVM no proporcionan directamente estimaciones de probabilidad, estas se calculan utilizando una costosa validación cruzada.    

In [ ]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_transform, y_train)

### Resultados obtenidos

In [ ]:
y_pred = clf.predict(X_test_tranform)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
plot_roc_curve(clf,X_test_tranform,y_test)

In [ ]:
from sklearn.metrics import plot_roc_curve,plot_confusion_matrix
plot_confusion_matrix(clf,X_test_tranform,y_test,normalize='true')

In [ ]:
features = pd.DataFrame()
features['feature'] = cv.get_feature_names()
features['importance'] = clf.coef_.flatten()

features = features.sort_values(by='importance')

features_top_positivas = features[:200]
features_top_negativas = features[-200:]

px.bar(pd.concat([features_top_negativas, features_top_positivas]).sort_values(by='importance'), x='feature', y='importance')

## Modelo RandomForest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train_transform, y_train)

### Resultados obtenidos

In [ ]:
y_pred = clf.predict(X_test_tranform)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import plot_roc_curve,plot_confusion_matrix,plot_precision_recall_curve
plot_confusion_matrix(clf,X_test_tranform,y_test,normalize='true')

In [ ]:
plot_roc_curve(clf,X_test_tranform,y_test)

In [ ]:
from sklearn.metrics import plot_precision_recall_curve

plot_precision_recall_curve(clf,X_test_tranform,y_test)

In [ ]:
features = pd.DataFrame()
features['feature'] = cv.get_feature_names()
features['importance'] = clf.feature_importances_

features = features.sort_values(by='importance',ascending=False)

px.bar(features.head(100), x='feature', y='importance')

## Posibles cambios
* Entrenar con lematizador o Stemmer
* Buscar los hiperónimo de cada palabra
* Eliminar símbolos, numeros, tildes, ñ o caracteres inválidos
* Entrenar diferentes tipos de modelos -> SVM, Naive Bayes, regresión logística
* Cross validation
* Incluir Bigramas y Trigramas
* Explorar técnicas para reducción de dimensionalidad y seleccion de features
* Regularizacion
* Vectorizacion de tokens con TF-IDF
